In [1]:
# Enter the path to the experiment folder eg r"F:\Light_curves\Experiment1"
path = r"F:\Light_curve_05_08_19\Experiment3"

# What name would you like to give to the results file? 
name = "Experiment3"

# Enter the pulse width (ms)
pulse_width = 300

# Do you have a report file in the path folder you wish to add the kinetics to? True or False (notice the capitals)
integrate_report = True

# what is the width of the initial slope? (ms)
fitting_window = 10

In [2]:


import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn import preprocessing
from scipy import stats

dir_list = os.listdir(path)




def Kinetics_Fitting(path, pulse_width = 300, integrate_report = False, Report_DF = None,fitting_window =10, plot_results=False):       

    FK_columns = ["P700 k","P700 r value", "P700 p value", "P700 std err", "PC k","PC r value", "PC p value", "PC std err","Fd k","Fd r value", "Fd p value", "Fd std err","P700 t1/2","PC t1/2","Fd t1/2"]
    FK_data = []
    for fk in os.listdir(path+"/FastKinetics"):
        fastkin_df = pd.read_csv(path+"/FastKinetics"+"/"+fk,sep=';',skiprows=1)


        cols_to_normalise = ["PC rel.","P700 rel.","Fd rel."]
        x = fastkin_df[cols_to_normalise[:]].values #returns a numpy array
        min_max_scaler = preprocessing.MinMaxScaler()
        x_scaled = min_max_scaler.fit_transform(x)
        normalised_df = pd.DataFrame(x_scaled,columns = cols_to_normalise)

        # DIRK initial rate (linear fit)
        DIRK_df = normalised_df[(fastkin_df["time/ms"]>=pulse_width) & (fastkin_df["time/ms"]<=pulse_width+fitting_window)]
        DIRK_df["time/ms"]= pd.Series([x/2.0 for x in range(len(DIRK_df.index))], index = DIRK_df.index)


        # Generated linear fit
        P700_slope, P700_intercept, P700_r_value, P700_p_value, P700_std_err = stats.linregress(np.array(DIRK_df["time/ms"]),np.array(DIRK_df["P700 rel."]))
        PC_slope, PC_intercept, PC_r_value, PC_p_value, PC_std_err = stats.linregress(np.array(DIRK_df["time/ms"]),np.array(DIRK_df["PC rel."]))
        Fd_slope, Fd_intercept, Fd_r_value, Fd_p_value, Fd_std_err = stats.linregress(np.array(DIRK_df["time/ms"]),np.array(DIRK_df["Fd rel."]))



        # Half time determination 
        DIRK_HALFTIME_df = normalised_df[(fastkin_df["time/ms"]>=pulse_width)]
        DIRK_HALFTIME_df["time/ms"]= pd.Series([x/2.0 for x in range(len(DIRK_HALFTIME_df.index))], index = DIRK_HALFTIME_df.index)

        # second normalisation and baseline correction

        cols_to_normalise = ["PC rel.","P700 rel.","Fd rel."]
        DIRK_HALFTIME_df["PC rel."] -= np.mean(np.array(DIRK_HALFTIME_df["PC rel."])[-100:])
        DIRK_HALFTIME_df["PC rel."] /= np.mean(np.array(DIRK_HALFTIME_df["PC rel."])[:3])

        DIRK_HALFTIME_df["P700 rel."] -= np.mean(np.array(DIRK_HALFTIME_df["P700 rel."])[-100:])
        DIRK_HALFTIME_df["P700 rel."] /= np.mean(np.array(DIRK_HALFTIME_df["P700 rel."])[:3])

        DIRK_HALFTIME_df["Fd rel."] = 1 - DIRK_HALFTIME_df["Fd rel."]
        DIRK_HALFTIME_df["Fd rel."] -= np.mean(np.array(DIRK_HALFTIME_df["Fd rel."])[-100:])
        DIRK_HALFTIME_df["Fd rel."] /= np.mean(np.array(DIRK_HALFTIME_df["Fd rel."])[:3])

        DIRK_HALFTIME_df["Fd rel."] = DIRK_HALFTIME_df.rolling(window=2).mean() # to smooth out the noisy Fd signal

        # Find Half-time (+/- 10 %)
        P700_HT = np.mean(np.array(DIRK_HALFTIME_df[(DIRK_HALFTIME_df["P700 rel."]<=0.55) & (DIRK_HALFTIME_df["P700 rel."]>=0.45)]["time/ms"]))
        PC_HT = np.mean(np.array(DIRK_HALFTIME_df[(DIRK_HALFTIME_df["PC rel."]<=0.55) & (DIRK_HALFTIME_df["PC rel."]>=0.45)]["time/ms"]))
        Fd_HT = np.mean(np.array(DIRK_HALFTIME_df[(DIRK_HALFTIME_df["Fd rel."]<=0.55) & (DIRK_HALFTIME_df["Fd rel."]>=0.45)]["time/ms"]))

        FK_data.append([P700_slope,P700_r_value, P700_p_value, P700_std_err,PC_slope,PC_r_value, PC_p_value, PC_std_err, Fd_slope,Fd_r_value, Fd_p_value, Fd_std_err, P700_HT, PC_HT, Fd_HT])
        
        if plot_results == True:
                # plot DIRK

                if not os.path.exists(path+"/DIRK_Halftime"):
                    os.makedirs(path+"/DIRK_Halftime")

                else:
                    pass

                for column in cols_to_normalise:
                    fig=plt.figure()
                    ax=fig.add_subplot(111)
                    ax.set_xlabel("Time (ms)")
                    plt.scatter(np.array(DIRK_HALFTIME_df["time/ms"]),np.array(DIRK_HALFTIME_df[column]),s=0.1,color='k')

                    plt.savefig(path+"/DIRK_Halftime/"+column+fk[:-4]+".png",format='png',dpi=600)
                    plt.clf()
                    plt.cla()
                    plt.close('all')


    if integrate_report == True: 
        DIRK_results_df = pd.DataFrame(columns = FK_columns, data = np.array(FK_data), index = Report_DF.index )
        results_df = pd.merge(Report_DF,DIRK_results_df,on = Report_DF.index)
        results_df = results_df[['Date', 'Time', 'Rec.Time', 'Action', 'ID', 'Name', 'Temp',
               'PAR', 'F(I)', "Fo,Fo'", "Fm,Fm'", 'FMTm', 'F', 'Y(II)', 'Y(4S)',
               'ETR(II)', 'Y(NO)', 'Y(NPQ)', 'NPQ', 'qN', 'qP', 'qL', 'F/Fm', "Fm'/Fm",
               'P700ox', "P700m,P700m'", 'Y(I)', 'Y(ND)', 'Y(NA)', 'ETR(I)', 'PCox',
               "PCm,PCm'", 'Rel PCox', "Rel PCm'", 'Fdred', "Fdm,Fdm'", 'Rel Fdred',
               "Rel Fdm'", "P700 k","P700 r value", "P700 p value", "P700 std err", 
                                 "PC k","PC r value", "PC p value", "PC std err","Fd k",
                                 "Fd r value", "Fd p value", "Fd std err","P700 t1/2","PC t1/2","Fd t1/2"]]
    else:
        results_df = pd.DataFrame(columns = FK_columns, data = np.array(FK_data))
    return results_df




In [3]:
# load report
for f in dir_list:
    if (f.endswith(".CSV") or f.endswith(".csv")) and not f.startswith("Results"):
        file_path = path+"/"+str(f)
        Report_DF = pd.read_csv(file_path,sep=';',skiprows=1)
        Report_DF = Report_DF[Report_DF["ID"]=="SP"]

        
results_df = Kinetics_Fitting(path, pulse_width = pulse_width, integrate_report = integrate_report, Report_DF = Report_DF,fitting_window =fitting_window)


# Make a time column in seconds/minutes from start
time_in_seconds = []
time_in_minutes = []
for i in results_df["Time"]:
    t = i.split(":")
    s = int(t[1])*60 + int(t[2])
    m = float(t[1]) + float(t[2])/60.0
    time_in_seconds.append(s)
    time_in_minutes.append(m)
results_df["Time/s"] = pd.Series(time_in_seconds,index=results_df.index)
results_df["Time/min"] = pd.Series(time_in_minutes,index=results_df.index)

# other engineered features (1-qL, YI/YII, ETR(I)-ETR(II))
results_df["1-qL"] = 1 - results_df["qL"]
results_df["Y(I)/Y(II)"] = results_df["Y(I)"] / results_df["Y(II)"]
results_df["ETR(I)-ETR(II)"] = results_df["ETR(I)"] - results_df["ETR(II)"]

results_df.to_csv(path+"/"+name+"_"+"Results.CSV")

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\sit

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\sit

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\sit

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\sit

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\sit

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\sit

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Anaconda\lib\sit

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
